In [1]:
from numpy import load, linspace, diag, eye, inf, fill_diagonal, dot, linalg
from SpiPy.Models.SAR import *
from DTO.Database import *

In [2]:
db_manager = HLDatabase()
pollution = db_manager.get_table(table_name='Train-Pollution')
pollution.set_index('Date', inplace=True, drop=True)

In [3]:
mSpatial = db_manager.get_table(table_name='Train-Weight Matrix')
tWind = load(r"../DTO/train_tWind.npy")
tZ = load(r"../DTO/train_tZ.npy")

In [4]:
pollution.values.shape

(13406, 7)

In [5]:
tZ.shape

(13406, 7, 7)

In [6]:
tWind.shape

(13406, 7, 7)

In [7]:
mSpatial = mSpatial.drop(labels='index', axis=1)

In [8]:
N = len(pollution.columns)
initial_params = [0.5]*N + [0.2] + [0.5] + [1]*N  
bounds = [(-1, 1)]*N + [(-1, 1)] + [(0, 1)] + [(-10000, 10000)]*N
zeta_values = linspace(0, 1, 11)

In [9]:
# Create an instance of the optimizer
optimizer = QMLEOptimizerWithZetaGridSearch(
    initial_params=initial_params,
    bounds=bounds,
    zeta_values=zeta_values,
    weight_matrix=mSpatial,
    wind_tensor=tWind,
    exog=pollution,
    Z=tZ
)

# Fit the model using grid search and maximum likelihood estimation
optimizer.fit()

# Retrieve the best parameters
best_params = optimizer.get_best_params()


print("Best Parameters:")
print("phi:", best_params[0])
print("alpha:", best_params[1])
print("rho:", best_params[2])
print("zeta:", best_params[3])
print("Sigma:", best_params[4])

C:\Users\VY72PC\PycharmProjects\Academia\Thesis-Code\SpiPy\Models\SAR.py:55: RuntimeWarning: divide by zero encountered in log
  component1 = -0.5*log(Sigma) + log(A_t_minus_1) - 0.5 * dot(residual_t.T, dot(linalg.inv(Sigma), residual_t))


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\VY72PC\AppData\Roaming\Python\Python310\site-packages\scipy\optimize\_differentiable_functions.py", line 141, in fun_wrapped
    fx = np.asarray(fx).item()
ValueError: can only convert an array of size 1 to a Python scalar

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\VY72PC\.conda\envs\Thesis-Code\lib\site-packages\IPython\core\interactiveshell.py", line 3505, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\VY72PC\AppData\Local\Temp\ipykernel_25024\4271825615.py", line 13, in <module>
    optimizer.fit()
  File "C:\Users\VY72PC\PycharmProjects\Academia\Thesis-Code\SpiPy\Models\SAR.py", line 69, in fit
  File "C:\Users\VY72PC\AppData\Roaming\Python\Python310\site-packages\scipy\optimize\_minimize.py", line 696, in minimize
    res = _minimize_lbfgsb(fun, x0, args, jac, bounds,
  File "C:\Users\VY72PC\AppData\Roaming\Python\P